In [15]:
using Random, GMRF

include("src/dataGen.jl");
include("src/model.jl");
include("src/utils.jl");

Taille de la grille (à ne pas modifier)

In [2]:
M₁ = 3;
M₂ = 3;
M = M₁ * M₂;

- Génération de la grille cible
- Création des données artificielles

In [3]:
Random.seed!(300);
Fmu = iGMRF(M₁, M₂, 1, 10);
Fphi = iGMRF(M₁, M₂, 1, 100);
gridTarget = generateTargetGrid(Fmu, Fphi);
gridTarget[:, :, 1] = gridTarget[:, :, 1] .+ 10.0;
nobs = 100;
data = generateData(gridTarget, nobs);

Réglages de CAVI

In [4]:
nEpoch = 2;
epochSize = 2;

Structure de stockage des traces

In [5]:
muMeanTrace = zeros(M, nEpoch*epochSize);
phiMeanTrace = zeros(M, nEpoch*epochSize);
xiMeanTrace = zeros(nEpoch*epochSize);
varTrace = zeros(M, nEpoch*epochSize, 9);
MCKL = zeros(nEpoch);

Initialisation  
On a seulement besoin d'initialiser les m  
Ceux-ci peuvent être initialisés à 0 -> on n'y touche pas

In [20]:
using ForwardDiff

for k = 1:nEpoch
    for j = 1:epochSize
        iter = (k - 1) * epochSize + j;
        if iter > 1
            muMeanTrace[:, iter] = muMeanTrace[:, iter-1];
            phiMeanTrace[:, iter] = phiMeanTrace[:, iter-1];
            xiMeanTrace[iter] = xiMeanTrace[iter-1];
            for i = 1:M
                μ̄i = neighborsMean(i, muMeanTrace[:, iter], Fmu);
                ϕ̄i = neighborsMean(i, phiMeanTrace[:, iter], Fphi);
                g(θi::DenseVector) = logfullconditional(
                    i,
                    θi,
                    μ=muMeanTrace[:, iter],
                    ϕ=phiMeanTrace[:, iter],
                    μ̄i=μ̄i,
                    ϕ̄i=ϕ̄i,
                    Fmu=Fmu,
                    Fphi=Fphi,
                    data=data,
                );
                (muMeanTrace[i, iter], phiMeanTrace[i, iter], xiMeanTrace[iter]) = findMode(
                    θi -> g(θi),
                    [muMeanTrace[i, iter], phiMeanTrace[i, iter], xiMeanTrace[iter]],
                );
                varTrace[i, iter, :] = flatten(fisherVar(
                    θi -> g(θi),
                    [muMeanTrace[i, iter], phiMeanTrace[i, iter], xiMeanTrace[iter]]
                ));
            end
            println(xiMeanTrace[iter])
            xiMeanTrace[iter] = findMode(
                ξ -> xilogfullconditional(
                    ξ,
                    μ=muMeanTrace[:, iter],
                    ϕ=phiMeanTrace[:, iter],
                    data=data,
                ),
                [xiMeanTrace[iter]],
            );
            println(xiMeanTrace[iter])
        end
    end
    approxMarginals = [
        MvNormal(
            [muMeanTrace[i, iter], phiMeanTrace[i, iter], xiMeanTrace[iter]],
            vecToMatrix(varTrace[i, iter, :]),
        )
        for i = 1:M
    ];
    # MCKL[k] = MonteCarloKL(logposterior, approxMarginals);
end

MethodError: MethodError: no method matching xilogfullconditional(::Vector{Float64}; μ::Vector{Float64}, ϕ::Vector{Float64}, data::Vector{Vector{Float64}})

Closest candidates are:
  xilogfullconditional(!Matched::Real; μ, ϕ, data)
   @ Main ~/Documents/dd/maitrise/recherche/code/autoCAVI/etape5/src/model.jl:83


In [25]:
size(tensor)

(3, 4, 5)